In [1]:
# First party libraries
import time
import csv
import os

In [2]:
# 3rd Party Libraries
import pandas as pd
import numpy as np

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By

### Need to set the below fields for race data to get and where to save results
- athlink: URL for race
- race_option: option under race for the desired distance. Option number currently matches order in Race drop down.
- out_fname: output file to write read in data to

In [3]:
athlink = 'https://www.athlinks.com/event/229325/results/Event/1039501/Course/2320851/Results'
# out_fname = 'Steamboat_15K_2023_results.csv'
out_fname = 'Steamboat_4mi_2023_results.csv'
race_option = 'option-2' #4 mile

# athlink = 'https://www.athlinks.com/event/229325/results/Event/1007071/Course/2203536/Results'
# out_fname = 'Steamboat_15K_2022_results.csv'
# out_fname = 'Steamboat_4mi_2022_results.csv'
# race_option = 'option-2' #4 mile

# athlink = 'https://www.athlinks.com/event/229325/results/Event/974250/Results'
# out_fname = 'Steamboat_4mi_2021_results.csv'
# race_option = 'option-1' #4 mile

# athlink = 'https://www.athlinks.com/event/229325/results/Event/861668/Results'
# out_fname = 'Steamboat_15K_2019_results.csv'
# out_fname = 'Steamboat_4mi_2019_results.csv'
# race_option = 'option-1' #4 mile

# athlink = 'https://www.athlinks.com/event/229325/results/Event/706436/Results'
# out_fname = 'Steamboat_15K_2018_results.csv'
# out_fname = 'Steamboat_4mi_2018_results.csv'
# race_option = 'option-1' #4 mile

# athlink = 'https://www.athlinks.com/event/229325/results/Event/648093/Results'
# out_fname = 'Steamboat_15K_2017_results.csv'
# out_fname = 'Steamboat_4mi_2017_results.csv'
# race_option = 'option-1' #4 mile

# athlink = 'https://www.athlinks.com/event/229325/results/Event/549744/Results'
# out_fname = 'Steamboat_4mi_2016_results.csv'
# race_option = 'option-3' #15K
# race_option = 'option-4' #4 mile


In [4]:
'''
Process passed in row and return dictionary of data
'''
def proc_row(ele):
    runner = {}
    resultCols = ele.find_elements(By.XPATH, './div')
    runnerDetElem = resultCols[0].find_elements(By.XPATH,'./div/div')[1]
    runner['name'] = runnerDetElem.find_element(By.TAG_NAME,'a').text
    
    runnerDetList = runnerDetElem.find_elements(By.TAG_NAME, 'span')
    gender_age = runnerDetList[0].text.split(' ')
    if len(gender_age) >=2:
        runner['gender'] = gender_age[0]
        runner['age'] = gender_age[1]
    else:
        runner['gender'] = gender_age[0]
    runner['bib'] = runnerDetList[1].text.split(' ')[1]
    
    runner['time'] = resultCols[5].text
    return runner

In [5]:
chrome_options = Options()
chrome_options.add_experimental_option('prefs', {
    'credentials_enable_service': False,
    'profile': {
        'password_manager_enabled': False
    }
})

browser = webdriver.Chrome(options=chrome_options)

# Open page and load race distance

In [6]:
browser.get(athlink)
time.sleep(5)

In [7]:
racesElem = browser.find_element(By.ID, 'race')
print(racesElem.text)


All Race Types


In [8]:
parentElem = racesElem.find_element(By.XPATH, "./..")
print(parentElem.get_attribute('innerHTML'))


<div class="MuiSelect-root MuiSelect-select MuiSelect-selectMenu MuiInputBase-input MuiInput-input" tabindex="0" role="button" aria-haspopup="listbox" aria-labelledby="race-label race" id="race">All Race Types</div><input aria-hidden="true" tabindex="-1" class="MuiSelect-nativeInput" value="0"><svg class="MuiSvgIcon-root MuiSelect-icon" focusable="false" viewBox="0 0 24 24" aria-hidden="true"><path d="M7 10l5 5 5-5z"></path></svg>


In [9]:
inputElem = parentElem.find_element(By.TAG_NAME, 'input')
parentElem.click()

In [10]:
# Option to use is based on order in the drop down list, which veries by event
raceTypeElem = browser.find_element(By.ID, race_option)
raceTypeElem.click()
time.sleep(3)

In [11]:
# some races can get the event ID from the URL but older ones do not
# event_id = athlink.split('/')[-2]
# if 'Division' in athlink:
#     event_id = athlink.split('/')[-4]
# # event_id = str(1611450) #Steamboat_15K_2019
# event_id = str(1152845) #Steamboat_15K_2018
# event_id = str(934543) #Steamboat_15K_2017
# event_id = str(818177) #Steamboat_15K_2016
# event_id = str(818176) #Steamboat_4mi_2016

updated_url = browser.current_url
print(updated_url)
event_id = updated_url.split('/')[-2]
print(event_id)


https://www.athlinks.com/event/229325/results/Event/1039501/Course/2320852/Results
2320852


# Get results

In [12]:
runners = []
pages_remain = True
while pages_remain:
    resultsParent = browser.find_element(By.ID, 'eventResults_'+event_id)
    # resultElemLst = resultsParent.find_elements(By.TAG_NAME, 'div')
    resultElemLst = resultsParent.find_elements(By.XPATH, './div')
    for resultElem in resultElemLst:
        runner = proc_row(resultElem)
        
        # print('Runner: ' + str(runner))
        # print(runner['name'] + ', ' + runner['gender'] + runner['age'] \
              # + ' Bib: ' + runner['bib']+ ' finished in ' + runner['time'])
        runners.append(runner)
    
    pager = browser.find_element(By.ID, 'pager')
    last_pager_button = pager.find_elements(By.TAG_NAME, 'button')[-1]
    # print('last button text:' + last_pager_button.text)
    if last_pager_button.text != '>':
        pages_remain = False
    else:
        browser.execute_script("arguments[0].scrollIntoView();", last_pager_button)
        last_pager_button.click()
        time.sleep(5)
        



If a runner crosses checkpoints but does not finish the race they may show at the end of the results with just their time up to that checkpoint. It will be under a different section but the job could pick those up if they are on a page just of people who did not finish the race. So need to check results at end and possibly remove those users. 

In [13]:
browser.quit()

# Save results to CSV

In [14]:
field_names = ['name', 'bib', 'gender', 'age', 'time']
home_dir = os.path.expanduser('~')
with open(os.path.join(home_dir, 'Downloads', out_fname),'w') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames = field_names)
    writer.writeheader()
    writer.writerows(runners)